# Pekiştirmeli Öğrenme

* Pekiştirmeli öğrenme, davranışçılıktan esinlenen, öznelerin bir ortamda en yüksek ödül miktarına ulaşabilmesi için hangi eylemleri yapması gerektiğiyle ilgilenen bir makine öğrenmesi yaklaşımıdır.
* Ajan olarak adlandırılan model, çevresiyle etkileşime girerek, daha önceden belirlenen ödül - ceza parametreleri sayesinde, problemin çözümü için gerekli olan eylemleri nasıl yapacağını öğrenir.
* Bu bölümde Pekiştirmeli Öğrenme'nin temellerinden olan Q -Öğrenme yöntemini kullanacağız.

## Durum, Eylem, Ödül, Deterministik ve Stokastik

* Q-öğrenmede amaç, maksimum değeri bularak ayrı bir eylemler dizisinden tek bir deterministik eylem öğrenmektir.
* Diğer bir deyişle ödülü en üst düzeye çıkaran en uygun yolu bulmak
* Kullanılan terimler:
    * Ajan: Çevrede eylem gerçekleştiren model.
    * Çevre: Ajanın eylemlerini gerçekleştirebildiği ortam, mesela 2 boyutlu bir oda.
    * Durum: Ajanın çevrede içerisinde bulunduğu durum, mesela 2 boyutlu bir odada x, y konumu.
    * Eylem: Hareket, mesela 2 boyutlu bir odada ileri ya da geri gitmek.
    * Ödül: Ajanın eylemleri sonucunda çevreden aldığı geri bildirim, mesela odada ulaşması gereken konuma ulaşınca aldığı +5 puan ya da odanın içerisinde çukura düşünce aldığı -5 puan.
    * Deterministik ve Stokastik: Ajanın hareketlerinin rastgelelik bulunmaması ya da rastgele olması

## Bellman Denklemi

* Belirli bir durumun değeri, belirli bir durumda optimum eylemin ödülünün maksimum eylemine eşittir ve bir indirim faktörü, Bellman Denkelmi'nden sonraki durumun değeri ile çarpılır.

## Markov Karar Süreci

* Markov özelligi, gelecegin gecmise degil, yalnizca bugüne bagli oldugunu belirtir.
* Markov zinciri, önceki durumlara degil, yalnizca mevcut duruma bagli olan, yani gelecek koşullu olarak geçmisten bagimsiz olan olasiliksal bir modeldir.
* MKS, Markov zincirinin bir uzantisidir. Karar verme durumlarini modellemek icin matematiksel bir çerceve saglar. Hemen hemen tüm Pekistirmeli Ögrenme problemleri MMKSDP olarak modellenebilir. MKS, 5 Onemli unsurla temsil edilebilir:
    1. Temsilcinin içinde olabilecegi bir dizi durum 
    2. Bir durumdan digerine geçmek icin bir aracı tarafindan gerçeklestirilebilecek bir dizi eylem 
    3. Bir eylem gerceklestirerek bir durumdan digerine geçme olasiligi olan bir geciş olasiligi 
    4. Bir eylemi gerçeklestirerek bir durumdan baska bir duruma geçmek için temsilci tarafindan elde edilen bir ödülün olasiligi olan ödül olasiligi
    5. Hemen ve gelecekteki ödülllerin önemini kontrol eden bir indirim faktörü 

## Donmuş Göl Uygulaması

* Donmuş Göl ortamı 4x4'lük bir ortamdır.
* 4 farklı alan bulunmaktadır.
    1. S: Başlangıç
    2. G: Hedef
    3. F: Güvenli Yol
    4. H: Delik
* 4 farklı eylem seçeneğimiz vardır:
    1. Sol
    2. Sağ
    3. İleri
    4. Geri
* Eğer ajan hedefe ulaşırsa ödül 1, ulaşamazsa ödül 0'dır.    

In [4]:
import gym
import numpy as np
import random
import matplotlib.pyplot as plt

In [6]:
# ortamı çağır
env = gym.make("FrozenLake-v1")

In [7]:
env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<FrozenLakeEnv<FrozenLake-v1>>>>>

In [8]:
# Q table oluştur
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [9]:
# Hiperparametreler
gamma = 0.95
alpha = 0.80
epsilon = 0.10

# Ödülleri görselleştirmek için bir liste oluştur
reward_list = []

In [12]:
# öğrenmeye başla

episode_number = 100000 # 100000 bölüm olsun
for i in range(1, episode_number):
    
    state = env.reset() # her bölümün başında ortamı resetle
    reward_count = 0
    
    while True:
        
        # %10 keşif, %90 sömürü oranı
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])
        
        next_state, reward, done, _ = env.step(action)
        
        # Q öğrenme fonksiyonu
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        next_value = (1-alpha)*old_value + alpha*(reward + gamma*next_max)
        
        # Q tablosunu güncelle
        q_table[state, action] = next_value
        
        # durum güncelle
        state = next_state
        
        # toplam ödülü hesapla
        reward_count += reward
        
        # eğer bölüm bittiyse kır ve yeni bölüme başla
        if done:
            break
        
    if i % 5000 == 0:
        print("Episode: {}".format(i))
    if i % 1000 == 0:
        reward_list.append(reward_count)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices